In [20]:
# import resnet 18
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import io
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.mixture import GaussianMixture
import numpy as np
import os
import csv
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from time import perf_counter


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

def read_image(image_path):
    img = io.read_image(image_path)

    weights = ResNet18_Weights.DEFAULT
    transform = weights.transforms()
    return transform(img)

def train(model, train_loader, optimizer, criterion):
    model.train()
    train_loss, correct = 0, 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()

    train_accuracy = correct / len(train_loader.dataset)
    train_loss /= len(train_loader)

    return train_loss, train_accuracy

def test(model, test_loader, criterion):
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()

    test_accuracy = correct / len(test_loader.dataset)
    test_loss /= len(test_loader)

    return test_loss, test_accuracy

def train_loop(images, num_epochs=15, batch_size=32, learning_rate=0.001, weight_decay=0.0001):
    # Data transformations
    weights = ResNet18_Weights.DEFAULT
    transform = weights.transforms()

    # Load dataset (assuming a folder structure compatible with torchvision.datasets.ImageFolder)
    dataset = datasets.ImageFolder(images, transform=transform)

    # Train-test split
    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
    train_set = Subset(dataset, train_idx)
    test_set = Subset(dataset, test_idx)

    # Data loaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4)

    # Load ResNet18 and modify the final layer
    resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
    for param in resnet.parameters():
        param.requires_grad = False
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, 6)  # Assuming 6 classes

    # Criterion and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Track metrics
    metrics = pd.DataFrame(columns=['epoch', 'train_loss', 'train_accuracy', 'test_loss', 'test_accuracy'])
    

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train(resnet, train_loader, optimizer, criterion)
        test_loss, test_accuracy = test(resnet, test_loader, criterion)

        # save model and metrics
        metrics[epoch] = [epoch, train_loss, train_accuracy, test_loss, test_accuracy]
        metrics.to_csv('metrics.csv', index=False)
        torch.save(resnet.state_dict(), 'model.pth')

        print(f"Epoch {epoch + 1}/{num_epochs}: "
              f"Train Loss={train_loss:.4f}, Train Acc={train_accuracy:.4f}, "
              f"Test Loss={test_loss:.4f}, Test Acc={test_accuracy:.4f}")


    print("Training complete!")

# Example usage
# train_loop('path/to/image/dataset', num_epochs=15)
